## make sure you have all libraries installed. 
use conda environment: /scratch/da2734/twitter/worldbank_twitter_environment.yml

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# loading filtered samples

In [ ]:
# filtered contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/filtered/'
tweets_filtered=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_filtered['tweet_id'] = tweets_filtered['tweet_id'].astype(int)
print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds')
print(tweets_filtered.shape)

In [ ]:
start_time = time.time()
tweets_filtered = tweets_filtered[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

In [ ]:
tweets_filtered.head()

# loading random samples

In [2]:
# random contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/random/'
tweets_random=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_random['tweet_id'] = tweets_random['tweet_id'].astype(int)
print('time taken to load random sample:', str(time.time() - start_time), 'seconds')
print(tweets_random.shape)

time taken to load random sample: 174.8864951133728 seconds
(92114009, 11)


In [3]:
start_time = time.time()
tweets_random = tweets_random[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

drop columns: 5.356445550918579 seconds


In [4]:
tweets_random.head()

,tweet_id,text
0,367881326273105920,@shoebydoo32 I only left to go back home for t...
1,367881326281519105,oh my god bina id idnt read that u played GTA ...
2,367882185916702722,I Have To Make What I Think Is The Best Decisi...
3,367883553121394690,@elizrod_ that's from hard work 😉
4,367883867719348224,RT @MileenaSucks: Can I just lay out in the gr...


In [ ]:
del merged_filtered

# merging random tweets and taking top 10%

In [10]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
#     if column != 'job_offer': continue
    
#     start_time = time.time()
    # model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/{}/'.format(column)
#     print(model_output_path)

    start_time = time.time()
    model_output_random1 = pd.concat([pd.read_csv(f) for f in glob.glob(
        '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert_inferences/inferences_dhaval/{}/'.format(column)+
        'random*.csv')], ignore_index = True)
    print('dhaval time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random1.shape)
    
    start_time = time.time()
    model_output_random2 = pd.concat([pd.read_csv(f) for f in glob.glob(
        '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert_inferences/inferences_sam/{}/'.format(column)+
        'random*.csv')], ignore_index = True)
    print('sam time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random2.shape)

    start_time = time.time()
    model_output_random = pd.concat([model_output_random1, model_output_random2])
    print('concat:', str(time.time() - start_time), 'seconds')

    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'first', 'second']]
    print('drop columns:', str(time.time() - start_time), 'seconds')    

    start_time = time.time()
    merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)

    # print(merged_random.head())

    start_time = time.time()
    merged_random = merged_random.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)
    
    start_time = time.time()
    merged_random = merged_random.sort_values(by=['first'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_random.shape)    
    
    start_time = time.time()
    merged_random_temp = merged_random[:5000000]
    print('top 10M truncate:', str(time.time() - start_time), 'seconds', merged_random.shape)   
    
    print(merged_random_temp.tail(50))
    
    start_time = time.time()
    pickle.dump( merged_random_temp, open( "./ONNXturk100M/iteration1/{}_ONNX_BERT_ST_merged_random_100m_jun22_first.pkl".format(column), "wb" ) )
    print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break




 is_unemployed
dhaval time taken to load random sample: 20.35538411140442 seconds (45812327, 3)
sam time taken to load random sample: 20.103055715560913 seconds (46272922, 3)
concat: 1.409827470779419 seconds
drop columns: 0.8331263065338135 seconds
time taken to merge: 95.40232038497925 seconds (92085249, 4)
drop dup: 119.44505739212036 seconds (84538243, 4)
sort: 59.434691429138184 seconds (84538243, 4)
top 10M truncate: 6.116557598114014 seconds (84538243, 4)
                    tweet_id     first    second  \
69279662  355409391308120065  0.987246  0.012754   
36961082  673245192007376896  0.987246  0.012754   
7205557   743589680654876672  0.987246  0.012754   
74288031  803382415934836738  0.987246  0.012754   
55433404  498411862430859264  0.987246  0.012754   
91117392  344741946624987137  0.987246  0.012754   
26064396  684221362408325120  0.987246  0.012754   
10749716  382021117730242561  0.987246  0.012754   
88039278  437706672924934144  0.987246  0.012754   
10815666  4

time taken to dump: 7.683283567428589 seconds


 job_search
dhaval time taken to load random sample: 74.74011969566345 seconds (45004464, 3)
sam time taken to load random sample: 87.79447603225708 seconds (45461959, 3)
concat: 1.3878576755523682 seconds
drop columns: 0.8168251514434814 seconds
time taken to merge: 115.46118569374084 seconds (90466423, 4)
drop dup: 131.2190020084381 seconds (83222049, 4)
sort: 61.294599294662476 seconds (83222049, 4)
top 10M truncate: 6.9155051708221436 seconds (83222049, 4)
                     tweet_id     first    second  \
76939138   980626219992481792  0.998065  0.001935   
78125836   507994349888892928  0.998065  0.001935   
20801405   588724303987892225  0.998065  0.001935   
23879949   779470437088538624  0.998065  0.001935   
46020844   523575614587293696  0.998065  0.001935   
6461924    421670321922768896  0.998065  0.001935   
40342145   611897116600840192  0.998065  0.001935   
3892906    850227489708195840  0.998065  0.001935   
37411146  

time taken to dump: 11.182528018951416 seconds


 job_offer
dhaval time taken to load random sample: 84.1784520149231 seconds (44836521, 3)
sam time taken to load random sample: 81.10985064506531 seconds (45336146, 3)
concat: 1.4327867031097412 seconds
drop columns: 1.3926692008972168 seconds
time taken to merge: 115.55263948440552 seconds (90172667, 4)
drop dup: 138.30413913726807 seconds (82963411, 4)
sort: 61.41395854949951 seconds (82963411, 4)
top 10M truncate: 6.598023176193237 seconds (82963411, 4)
                    tweet_id     first    second  \
19098327  323818304596758528  0.996078  0.003922   
35035005  910366540310118401  0.996078  0.003922   
44781678  458854391614038016  0.996078  0.003922   
50446379  665987565384810498  0.996078  0.003922   
26748742  498904974290468864  0.996078  0.003922   
36656284  680786408135614464  0.996078  0.003922   
6513785   727579883744808960  0.996078  0.003922   
86124506  502159485830713346  0.996078  0.003922   
77032744  35735965868

In [11]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
#     if column != 'job_offer': continue
    
#     start_time = time.time()
    # model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_ONNX_optimized/{}/'.format(column)
#     print(model_output_path)

    start_time = time.time()
    model_output_random1 = pd.concat([pd.read_csv(f) for f in glob.glob(
        '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert_inferences/inferences_dhaval/{}/'.format(column)+
        'random*.csv')], ignore_index = True)
    print('dhaval time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random1.shape)
    
    start_time = time.time()
    model_output_random2 = pd.concat([pd.read_csv(f) for f in glob.glob(
        '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert_inferences/inferences_sam/{}/'.format(column)+
        'random*.csv')], ignore_index = True)
    print('sam time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random2.shape)

    start_time = time.time()
    model_output_random = pd.concat([model_output_random1, model_output_random2])
    print('concat:', str(time.time() - start_time), 'seconds')

    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'first', 'second']]
    print('drop columns:', str(time.time() - start_time), 'seconds')    

    start_time = time.time()
    merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)

    # print(merged_random.head())

    start_time = time.time()
    merged_random = merged_random.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)
    
    start_time = time.time()
    merged_random = merged_random.sort_values(by=['second'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_random.shape)    
    
    start_time = time.time()
    merged_random_temp = merged_random[:5000000]
    print('top 10M truncate:', str(time.time() - start_time), 'seconds', merged_random.shape)   
    
    print(merged_random_temp.tail(50))
    
    start_time = time.time()
    pickle.dump( merged_random_temp, open( "./ONNXturk100M/iteration1/{}_ONNX_BERT_ST_merged_random_100m_jun22_second.pkl".format(column), "wb" ) )
    print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break




 is_unemployed
dhaval time taken to load random sample: 31.735576629638672 seconds (45812327, 3)
sam time taken to load random sample: 19.609444856643677 seconds (46272922, 3)
concat: 1.4148964881896973 seconds
drop columns: 0.836737871170044 seconds
time taken to merge: 93.29606199264526 seconds (92085249, 4)
drop dup: 126.9180371761322 seconds (84538243, 4)
sort: 61.13146233558655 seconds (84538243, 4)
top 10M truncate: 6.228193283081055 seconds (84538243, 4)
                     tweet_id     first    second  \
52529990   747418400402579456  0.833263  0.166737   
454976     673907724011311106  0.833263  0.166737   
52725584   333814035646656512  0.833263  0.166737   
78787381   296262872361750528  0.833263  0.166737   
24680734   685281527299575808  0.833263  0.166737   
91284096   388133980341411840  0.833263  0.166737   
66811300   468550217441099777  0.833263  0.166737   
58961344   551090437398990848  0.833263  0.166737   
33827504   353438152402284544  0.833263  0.166737   
52

time taken to dump: 4.537342309951782 seconds


 job_search
dhaval time taken to load random sample: 69.86781096458435 seconds (45004464, 3)
sam time taken to load random sample: 20.746444702148438 seconds (45461959, 3)
concat: 1.391749382019043 seconds
drop columns: 0.8277606964111328 seconds
time taken to merge: 114.85198307037354 seconds (90466423, 4)
drop dup: 140.02991557121277 seconds (83222049, 4)
sort: 62.646331787109375 seconds (83222049, 4)
top 10M truncate: 7.111707448959351 seconds (83222049, 4)
                     tweet_id     first    second  \
63735127   718263114068836354  0.990157  0.009843   
41415417   322469869167726592  0.990157  0.009843   
36070910   367751264794607616  0.990157  0.009843   
36824439   673723015373316096  0.990157  0.009843   
62330473   407246286782357504  0.990157  0.009843   
60126521   351380305015349248  0.990157  0.009843   
70528590   385995673625456640  0.990157  0.009843   
82235702   603351326978781185  0.990157  0.009843   
39228053  

time taken to dump: 4.209624528884888 seconds


 job_offer
dhaval time taken to load random sample: 23.794333696365356 seconds (44836521, 3)
sam time taken to load random sample: 23.147194385528564 seconds (45336146, 3)
concat: 1.389491081237793 seconds
drop columns: 0.834505558013916 seconds
time taken to merge: 98.66617941856384 seconds (90172667, 4)
drop dup: 130.70891118049622 seconds (82963411, 4)
sort: 71.70319652557373 seconds (82963411, 4)
top 10M truncate: 6.621304988861084 seconds (82963411, 4)
                     tweet_id     first    second  \
7712118    595483592668610560  0.963027  0.036973   
58454365   597087817542475776  0.963027  0.036973   
25445868   732479602677817344  0.963027  0.036973   
89154443   716991615022174208  0.963027  0.036973   
22469503   290200811252899840  0.963027  0.036973   
35916877   632952635872690176  0.963027  0.036973   
52484037   398819766636269569  0.963027  0.036973   
81995331   619947126617952256  0.963027  0.036973   
7052034    77